In [13]:
import synapseclient
import sys
import os
import pandas
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [14]:
sys.path.append("/Users/apratap/dev/appys/lib/")

In [15]:
import synUtils

In [16]:
syn = synapseclient.login()

Welcome, Abhishek Pratap!



In [17]:
elevateMS_PROJECT_SYNID = 'syn9746170'

In [18]:
tables = syn.query("select id,name from table where parentId =='%s'" % elevateMS_PROJECT_SYNID)
tables = pandas.DataFrame.from_dict(tables['results'])

In [19]:
def count_rows(tableId):
    x = syn.tableQuery("select * from %s" % tableId)
    return(x.asDataFrame().shape[0])
tables['numRecords'] = tables['table.id'].apply(count_rows)
tables

,table.id,table.name,numRecords
0,syn9758009,elevate-ms-appVersion,558
1,syn9758010,elevate-ms-Daily Check-In-v1,73
2,syn9758011,elevate-ms-status,61
3,syn9765504,elevate-ms-Tapping Activity-v1,117
4,syn9765702,elevate-ms-Symptoms Survey-v1,135
5,syn9819608,elevate-ms-Factors Survey-v1,51
6,syn9872551,elevate-ms-Relapses Survey-v1,17
7,syn9877098,elevate-ms-Medication Survey-v1,12
8,syn9890446,elevate-ms-Relapse Survey-v1,1
9,syn9920297,elevate-ms-Tremor Activity-v4,26


In [10]:
grp = tables.groupby(['table.name'])

In [12]:
grp

#### Select the latest version for each activity

In [9]:

selected_tables = grp.apply(lambda x: x.ix[ x.version == max(x.version), ['table.id', 'version'] ])
selected_tables = selected_tables.reset_index()
selected_tables = selected_tables.ix[:,['table_name', 'table.id', 'version']]
selected_tables

/Users/apratap/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:2: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate_ix
  from ipykernel import kernelapp as app


AttributeError: 'DataFrame' object has no attribute 'version'

### Create Wiki  & Table for the latest data

In [10]:
schema = synapseclient.Schema(name='Latest Data', columns=synapseclient.as_table_columns(selected_tables), parent=LILLY_PROJECT_SYNID)
uploaded_table= syn.store(synapseclient.Table(schema, selected_tables))

Uploading [####################]100.00%     209.0bytes/209.0bytes table.csv Done...
 [####################]100.00%     1/1  Done...
    

In [11]:
uploaded_table.tableId

u'syn7068542'

In [12]:
suffix = '##### This table can also be programmatically accessed from %s' % uploaded_table.tableId
synUtils.push_DF_to_Wiki(selected_tables, 'syn6101466', '401496', syn, suffixText=suffix )

Downloaded   [####################]100.00%     192.0bytes/192.0bytes 401496_markdown.txt Done...
    

u'|table-name|table.id|version|\n|--|--|--|\n|Medication Tracker|syn6187112|3|\n|Memory Activity|syn6187120|7|\n|Study Drug Tracker|syn6828586|1|\n|Tapping Activity|syn6187115|5|\n|Voice Activity|syn6187122|5|\n|Walking Activity|syn7058430|6|\n##### This table can also be programmatically accessed from syn7068542\n'